<a href="https://colab.research.google.com/github/ROCKMOHAN/OnMyWay2DeepLearning/blob/master/3C_Yolo_Mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda, GlobalAveragePooling2D
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 256
num_classes = 10
epochs = 50
l = 10
num_filter = 20

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1], x_train.shape[2], x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

#Has 4 Blocks with 1st Block to 4th Block Skip connection.

In [0]:
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer1 = BatchNormalization(name='norm_1')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
#layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)

# Layer 2
layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer2 = BatchNormalization(name='norm_2')(layer2)
layer2 = LeakyReLU(alpha=0.1)(layer2)
#layer2 = MaxPooling2D(pool_size=(2, 2))(layer2)

# Layer 3
layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer2)
layer3 = BatchNormalization(name='norm_3')(layer3)
layer3 = LeakyReLU(alpha=0.1)(layer3)

# Layer 4
layer4 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer3)
layer4 = BatchNormalization(name='norm_4')(layer4)
layer4 = LeakyReLU(alpha=0.1)(layer4)

bottleNeck1 = Conv2D(32, (1, 1), strides=(1,1), padding='same', name='bottleneck1', use_bias=False)(layer4)

# Layer 5
layer5 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(bottleNeck1)
layer5 = BatchNormalization(name='norm_5')(layer5)
layer5 = LeakyReLU(alpha=0.1)(layer5)

skipconnection = layer5

layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)

#Block 1 ends

#print(skipconnection.shape)

# Layer 6
layer6 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(layer5)
layer6 = BatchNormalization(name='norm_6')(layer6)
layer6 = LeakyReLU(alpha=0.1)(layer6)

# Layer 7
layer7 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer6)
layer7 = BatchNormalization(name='norm_7')(layer7)
layer7 = LeakyReLU(alpha=0.1)(layer7)

# Layer 8
layer8 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer7)
layer8 = BatchNormalization(name='norm_8')(layer8)
layer8 = LeakyReLU(alpha=0.1)(layer8)
#layer8 = MaxPooling2D(pool_size=(2, 2))(layer8)

# Layer 9
layer9 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer8)
layer9 = BatchNormalization(name='norm_9')(layer9)
layer9 = LeakyReLU(alpha=0.1)(layer9)

bottleNeck2 = Conv2D(32, (1, 1), strides=(1,1), padding='same', name='bottleneck2', use_bias=False)(layer9)


# Layer 10
layer10 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_10', use_bias=False)(bottleNeck2)
layer10 = BatchNormalization(name='norm_10')(layer10)
layer10 = LeakyReLU(alpha=0.1)(layer10)
layer10 = MaxPooling2D(pool_size=(2, 2))(layer10)

#Block 2 ends

# Layer 11
layer11 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer10)
layer11 = BatchNormalization(name='norm_11')(layer10)
layer11 = LeakyReLU(alpha=0.1)(layer10)

# Layer 12
layer12 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_12', use_bias=False)(layer11)
layer12 = BatchNormalization(name='norm_12')(layer12)
layer12 = LeakyReLU(alpha=0.1)(layer12)

# Layer 13
layer13 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(layer12)
layer13 = BatchNormalization(name='norm_13')(layer13)
layer13 = LeakyReLU(alpha=0.1)(layer13)


# Layer 14
layer14 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer13)
layer14 = BatchNormalization(name='norm_14')(layer14)
layer14 = LeakyReLU(alpha=0.1)(layer14)

bottleNeck3 = Conv2D(32, (1, 1), strides=(1,1), padding='same', name='bottleneck3', use_bias=False)(layer14)


# Layer 15
layer15 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_15', use_bias=False)(bottleNeck3)
layer15 = BatchNormalization(name='norm_15')(layer15)
layer15 = LeakyReLU(alpha=0.1)(layer15)
layer15 = MaxPooling2D(pool_size=(2, 2))(layer15)

#Block 3 ends

# Layer 16
layer16 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(layer15)
layer16 = BatchNormalization(name='norm_16')(layer16)
layer16 = LeakyReLU(alpha=0.1)(layer16)

# Layer 17
layer17 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_17', use_bias=False)(layer16)
layer17 = BatchNormalization(name='norm_17')(layer17)
layer17 = LeakyReLU(alpha=0.1)(layer17)

# Layer 18
layer18 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(layer17)
layer = BatchNormalization(name='norm_18')(layer18)
layer = LeakyReLU(alpha=0.1)(layer18)

# Layer 19
layer19 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(layer18)
layer19 = BatchNormalization(name='norm_19')(layer19)
layer19 = LeakyReLU(alpha=0.1)(layer19)

bottleNeck4 = Conv2D(32, (1, 1), strides=(1,1), padding='same', name='bottleneck4', use_bias=False)(layer19)


# Layer 20
layer20 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(bottleNeck4)
layer20 = BatchNormalization(name='norm_20')(layer20)
layer20 = LeakyReLU(alpha=0.1)(layer20)
layer20 = MaxPooling2D(pool_size=(2, 2))(layer20)

# Layer 21
skipconnection = Conv2D(128, (1,1), strides=(1, 1), padding='same', name='conv_21', use_bias=False)(skipconnection)
skipconnection = BatchNormalization(name='norm_21')(skipconnection)
skipconnection = LeakyReLU(alpha=0.1)(skipconnection)
skipconnection = Lambda(space_to_depth_x2)(skipconnection)

layer21 = concatenate([skipconnection, layer5])

# Layer 22
layer22 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(layer21)
layer22 = BatchNormalization(name='norm_22')(layer21)
layer22 = LeakyReLU(alpha=0.1)(layer21)

# Layer 23
#layer23 = Flatten()(layer22)

#output = Dense(num_classes, activation='softmax')(layer23)
output = Conv2D(10, (1, 1), activation='relu')(layer22)

output = GlobalAveragePooling2D()(output)
output = Activation('softmax')(output)

## Modified the network to have 10 output depth & used `GlobalAveragePooling2D` to get rid of **DenseLayer** - Reduced More that 2M Parameters

In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 32, 32, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 32, 32, 32)   128         conv_1[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu_67 (LeakyReLU)      (None, 32, 32, 32)   0           norm_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Co

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 145s 3ms/step - loss: 0.8641 - acc: 0.6952 - val_loss: 1.1047 - val_acc: 0.6326
Epoch 2/50
50000/50000 [==============================] - 143s 3ms/step - loss: 0.7877 - acc: 0.7228 - val_loss: 1.5024 - val_acc: 0.5263
Epoch 3/50
50000/50000 [==============================] - 143s 3ms/step - loss: 0.7375 - acc: 0.7408 - val_loss: 1.3454 - val_acc: 0.5717
Epoch 4/50
50000/50000 [==============================] - 143s 3ms/step - loss: 0.6844 - acc: 0.7599 - val_loss: 2.3965 - val_acc: 0.4154
Epoch 5/50
50000/50000 [==============================] - 143s 3ms/step - loss: 0.6444 - acc: 0.7755 - val_loss: 2.1986 - val_acc: 0.4631
Epoch 6/50
50000/50000 [==============================] - 142s 3ms/step - loss: 0.6021 - acc: 0.7909 - val_loss: 1.3599 - val_acc: 0.6121
Epoch 7/50
50000/50000 [==============================] - 142s 3ms/step - loss: 0.5744 - acc: 0.8004 - val_loss: 1.1894 - 

### Accuracy is 98.7 & Validation Accuracy is 77.5 on 50 Epochs

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 11s 1ms/step
Test loss: 0.977196123123169
Test accuracy: 0.7751
Saved the model to disk


In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model.h5')